<a href="https://colab.research.google.com/github/Jasper-Hewitt/Wind_project/blob/main/low_tensor_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## get and split data

In [ ]:
#todo
#fix the max number of tokens, it's actually very little
#補充 answer with the rest of the rows
#make a better while loop that creates different dataframes based on word count, code now a bit ugly
!pip install fitz
!pip install static
!pip install frontend
!pip install pymupdf --upgrade
!pip install openai

In [2]:
import os
import openai
import json

openai.api_key = '' #delete when uploading 

In [3]:
#manually download the data from github and upload it in this notebook
import fitz

doc = fitz.open("dragged_15_37.pdf")

for page in doc:
    text = page.get_text("text")
    with open("output3.txt", "a") as f:
        f.write(text)

In [ ]:
with open('output3.txt', 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

In [5]:
import re
pattern = "。" #。|:
split_contents = re.split(pattern, contents)
# print(sentences)

In [ ]:
split_contents

In [7]:
len(split_contents)

218

## sentiment search

In [ ]:
#sentiment search
!pip install sentence_transformers

from sentence_transformers import SentenceTransformer, util
import torch

embedder = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
corpus = split_contents
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# add queries, between 3 and 5
queries = ['對於鯨豚保育的重點是什麼？']

# Find the closest X sentences of the corpus for each query sentence based on cosine similarity
top_k = min(68, len(corpus))

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop X most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("Sentence: ", corpus[idx], " Original Sequence: ", idx+1, " (Score: {:.4f})".format(score))

In [9]:
len(queries)

1

In [74]:
sentence_scores = {sentence: {query: 0 for query in queries} for sentence in corpus}

for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=len(corpus))
    for score, idx in zip(top_results[0], top_results[1]):
        sentence = corpus[idx]
        sentence_scores[sentence][query] = score

In [75]:
len(sentence_scores)

198

In [76]:
#adjust to also allow for queries outside the range of 1-5
import pandas as pd

if len(queries)==1:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]]))

elif len(queries)==2:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[0]]+scores[queries[1]]))

elif len(queries)==3:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]))
elif len(queries)==4:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]))
elif len(queries)==5:
  results = []
  for i, (sentence, scores) in enumerate(sentence_scores.items()):
      results.append((sentence, i+1, scores[queries[0]], scores[queries[1]], scores[queries[2]], scores[queries[3]], scores[queries[4]], scores[queries[0]]+scores[queries[1]]+scores[queries[2]]+scores[queries[3]]+scores[queries[4]])) 

#________________________________________________________________________________________________________________________________________________________________________________________________________

if len(queries) == 1:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 2:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

elif len(queries) == 3:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 4:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)
elif len(queries) == 5:
    df = pd.DataFrame(results, columns=['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'similarity_score4', 'similarity_score5', 'accumulated_score'])
    df = df.sort_values(by='accumulated_score', ascending=False)

In [77]:
df

,text,original_sequence,accumulated_score
17,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...,18,"tensor(0.5316, device='cuda:0')"
180,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...,181,"tensor(0.4650, device='cuda:0')"
156,」因此本計畫位於彰化縣\n海域管轄範圍,157,"tensor(0.3782, device='cuda:0')"
132,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...,133,"tensor(0.3517, device='cuda:0')"
123,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...,124,"tensor(0.3516, device='cuda:0')"
...,...,...,...
59,\np.5-6 \n東西向風機間距約介於 533~779 公尺，南北向風機間距約介\n於 ...,60,"tensor(-0.0496, device='cuda:0')"
58,\n \n \n8 \n25.0(\nMSL) \n25.0(\nMSL)\n總高/ (葉...,59,"tensor(-0.0839, device='cuda:0')"
56,\np.5-6 \n表 5.2.1-1 風機設計包絡表 \n元件 \n最小風機 \n最大風...,57,"tensor(-0.1129, device='cuda:0')"
3,當選用單機裝置容量最小(4MW)\n的風機時，將設置 140 部風機；當選用單機裝置容量最大...,4,"tensor(-0.1296, device='cuda:0')"


In [78]:
noise_pattern= '遵照辦理'
noise_pattern1='章節'
noise_pattern2='��'
noise_pattern3='�'
noise_pattern4='修訂處'
noise_pattern5='頁次'

df['text'] = df['text'].replace(noise_pattern,'',regex=True)
df['text'] = df['text'].replace(noise_pattern1,'',regex=True)
df['text'] = df['text'].replace(noise_pattern2,'',regex=True)
df['text'] = df['text'].replace(noise_pattern3,'',regex=True)
df['text'] = df['text'].replace(noise_pattern4,'',regex=True)
df['text'] = df['text'].replace(noise_pattern5,'',regex=True)

In [79]:
#delete empty rows
df = df[df['text'].str.len()>6]   
#the line below no longer makes sense lol. 
# df = df[['text', 'original_sequence', 'similarity_score1', 'similarity_score2', 'similarity_score3', 'accumulated_score']]
df = df[['text', 'original_sequence', 'accumulated_score']]
df = df.reset_index(drop=True)
df

,text,original_sequence,accumulated_score
0,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...,18,"tensor(0.5316, device='cuda:0')"
1,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...,181,"tensor(0.4650, device='cuda:0')"
2,」因此本計畫位於彰化縣\n海域管轄範圍,157,"tensor(0.3782, device='cuda:0')"
3,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...,133,"tensor(0.3517, device='cuda:0')"
4,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...,124,"tensor(0.3516, device='cuda:0')"
...,...,...,...
192,\np.5-6 \n東西向風機間距約介於 533~779 公尺，南北向風機間距約介\n於 ...,60,"tensor(-0.0496, device='cuda:0')"
193,\n \n \n8 \n25.0(\nMSL) \n25.0(\nMSL)\n總高/ (葉...,59,"tensor(-0.0839, device='cuda:0')"
194,\np.5-6 \n表 5.2.1-1 風機設計包絡表 \n元件 \n最小風機 \n最大風...,57,"tensor(-0.1129, device='cuda:0')"
195,當選用單機裝置容量最小(4MW)\n的風機時，將設置 140 部風機；當選用單機裝置容量最大...,4,"tensor(-0.1296, device='cuda:0')"


In [80]:
#I used sentiment search to judge whether GPTs output is related to our keywords but I ran into a problem: there is no way to check whether the information is complete. For example, if we set the default threshold
#to 0.7000, and then ask chatGPT to make a summary about certain topics within that text, it will only make a summary about those topics and leave the rest#####
#also, if start with a really high threshold (which may only lead to 2 or 3 sentences) chat gpt will write a perfect summary that SBERT will judge with a high tensor score. This means that the threshold will not go down
#and much information that may also be related to the subject is being left out.

#I therefore thought of an alternative way to create a dynamic threshold. In the code below we find the last row, so the lowest possible tensor score, that contains our 關鍵詞 and delete all the rows after that.
#this way we can be sure that all the important sentences related to our topic will be taken into consideration by GPT for writing its summary or answering our questions. 

# Find the index of the last row containing the word "鯨豚"
index = df[df['text'].str.contains('豚'or'噪音')][::-1].index[0]

# Drop all of the rows after the last row containing the word "鯨豚"
new_df = df.loc[:index]

In [81]:
new_df

,text,original_sequence,accumulated_score
0,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...,18,"tensor(0.5316, device='cuda:0')"
1,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...,181,"tensor(0.4650, device='cuda:0')"
2,」因此本計畫位於彰化縣\n海域管轄範圍,157,"tensor(0.3782, device='cuda:0')"
3,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...,133,"tensor(0.3517, device='cuda:0')"
4,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...,124,"tensor(0.3516, device='cuda:0')"


## determine threshold

In [ ]:
#we will have to consider how we want to determine the threshold, the simplest way would be to look at the dataframe and check from where it doesn't have anything to do 
#with 噪音 anymore. if there is a more mathematical way to solve this problem I am also open to that. 

# threshold = 0.7710
# mask = df['accumulated_score'].apply(lambda x: x.item() > threshold)

# sbert_out = df[mask]
# sbert_out

## sort by original sequence (optional)

In [ ]:
# sbert_out = sbert_out.sort_values(by='original_sequence').reset_index(drop=True)
# sbert_out=sbert_out['text']

In [ ]:
# sbert_out=sbert_out.to_frame()
# sbert_out

## Summary with GPT 

### prepare data

In [82]:
#disable this cell for sort by orginal sequence 
#fix this this is super confusing lol!!!!
sbert_out=new_df
sbert_out=sbert_out['text']  
sbert_out=sbert_out.to_frame() 
df=sbert_out

In [83]:
df

,text
0,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...
1,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...
2,」因此本計畫位於彰化縣\n海域管轄範圍
3,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...
4,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...


In [84]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 1500
index = df.index[df['cumulative_length'] > 1500]

if index.size > 0:
    index = index[0] - 1
else:
    index = df.shape[0] - 1

# Create a new dataframe with only the last few rows
new_df = df.iloc[index+1:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index+1]
df

,text,text_length,cumulative_length
0,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...,58,58
1,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...,72,130
2,」因此本計畫位於彰化縣\n海域管轄範圍,18,148
3,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...,76,224
4,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...,167,391


In [85]:
sbert_out=df['text']
# sbert_out_1=new_df['text']
# sbert_out_1 = sbert_out_1.reset_index(drop=True)
# sbert_out.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
# sbert_out_1.loc[0] = ['第327次會議是什麼時候？ 第327次會議中，對於鯨豚保育的重點是什麼？ 第327次會議中，有哪些專家為鯨豚保育發言？']
sbert_out=sbert_out.to_frame()


In [86]:
sbert_out

,text
0,本潛力場址區域不包含漁港、濕地、保護礁區、漁\n業資源保育區、重要野鳥棲地、中華白海豚野生動...
1,然開發同時為兼顧再生能源發展及整體海岸管\n理，已考量整體生態保育、景觀、環境等因素，使海\...
2,」因此本計畫位於彰化縣\n海域管轄範圍
3,\np.6-9 \n十、全國區域計畫 \n… \n(四)與開發行為之關聯性 \n本計畫風機...
4,\n \n \n17 \n第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風\n力發電...


In [87]:
# sbert_text = sbert_out['text'].to_string(index=False) #this leads to incomplete text. use join
sbert_text = " ".join(sbert_out['text'].tolist())
len(sbert_text)

395

In [88]:
sbert_text = re.sub(r'[\n\\]+', '', sbert_text)
sbert_text= re.sub(r'n|n－', '', sbert_text)
sbert_text

'本潛力場址區域不包含漁港、濕地、保護礁區、漁業資源保育區、重要野鳥棲地、中華白海豚野生動物重要棲息環境…等限制區 然開發同時為兼顧再生能源發展及整體海岸管理，已考量整體生態保育、景觀、環境等因素，使海岸功能及國土保安皆能落實，創造海岸管理與能源轉型雙贏 」因此本計畫位於彰化縣海域管轄範圍  p.6-9 十、全國區域計畫 … (四)與開發行為之關聯性 本計畫風機設置區域並無位於全國區域計畫海域利用所述之彰雲嘉沿海保護區計畫範圍內    17 第二期計畫」「中能離岸風力發電開發計畫」「王功與永興風力發電計畫」「海峽離岸風力發電計畫(27 號風場)「海峽離岸風力發電計畫(28 號風場)」「彰化西島離岸風力發電計畫」「彰化彰芳離岸風力發電計畫」「彰化福芳離岸風力發電計畫」「中華白海豚野生動物重要棲息環境之類別及範圍(預告訂定)」等相關計畫'

In [89]:
len(sbert_text)

370

### prompt GPT

In [91]:
summary = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '對於鯨豚保育的重點是什麼？'+sbert_text,
  temperature=0.9,
  max_tokens=800,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary = summary + text

summary

'。\n\n鯨豚保育的重點一般包括：1. 減少受環境污染的損害；2. 減少經濟活動對鯨豚的衝擊；3. 保護重要漁業資源和野鳥棲地；4. 保護和修復重要海洋生態系統；5. 加強鯨豚及海洋生物研究；6. 加強對鯨豚的監管；7. 建立保護性地區與措施；8. 發展鯨豚攝影和觀賞活動的指標和管理方案；9. 推動海洋生態教育和環境意識；10. 提高民眾對鯨豚保護的參與度等。'

In [ ]:
# summary1 = ''

# response = openai.Completion.create(
#   model="text-davinci-003",
#   prompt= ' 第327次會議中，對於鯨豚保育的重點是什麼？'+sbert_text,
#   temperature=0.9,
#   max_tokens=500,
#   top_p=1,
#   frequency_penalty=0.0,
#   presence_penalty=0.0,
#   stop=[" Human:", " AI:"]
#   )
# response
# json_object = json.loads(str(response))
# text = json_object['choices'][0]['text']
# summary1 = summary1 + text

# summary1

In [ ]:
#  #remove key when uploading to github
# summary2 = ''

# response = openai.Completion.create(
#   model="text-davinci-003",
#   prompt= '第327次會議中，有哪些專家為鯨豚保育發言？'+sbert_text,
#   temperature=0.9,
#   max_tokens=500,
#   top_p=1,
#   frequency_penalty=0.0,
#   presence_penalty=0.0,
#   stop=[" Human:", " AI:"]
#   )
# response
# json_object = json.loads(str(response))
# text = json_object['choices'][0]['text']
# summary2 = summary2 + text

# summary2

In [92]:
print('第327次會議是什麼時候？'+ summary)
# print('第327次會議中，對於鯨豚保育的重點是什麼？'+ summary1)
# print('第327次會議中，有哪些專家為鯨豚保育發言？'+ summary2)

第327次會議是什麼時候？。

鯨豚保育的重點一般包括：1. 減少受環境污染的損害；2. 減少經濟活動對鯨豚的衝擊；3. 保護重要漁業資源和野鳥棲地；4. 保護和修復重要海洋生態系統；5. 加強鯨豚及海洋生物研究；6. 加強對鯨豚的監管；7. 建立保護性地區與措施；8. 發展鯨豚攝影和觀賞活動的指標和管理方案；9. 推動海洋生態教育和環境意識；10. 提高民眾對鯨豚保護的參與度等。


## adjust answers based on more information 

### prepare additional data

In [ ]:
new_df

In [ ]:
# 補充answer with other rows?
# reset index
new_df=new_df['text']
new_df = new_df.reset_index(drop=True)

In [ ]:
new_df=new_df.to_frame()

In [ ]:
df=new_df

In [ ]:
# Find the number of characters in each row of the text column
df['text_length'] = df['text'].str.len()

# Get the cumulative sum of the text lengths
df['cumulative_length'] = df['text_length'].cumsum()

# Find the index of the first row in which the cumulative length is greater than 3600
index = df.index[df['cumulative_length'] > 1700][0]

# Create a new dataframe with only the last few rows
new_df = df.iloc[index:]

# Drop the last few rows from the original dataframe
df = df.iloc[:index]

In [ ]:
df

In [ ]:
sbert_out_1=df['text']
sbert_out_1=sbert_out_1.to_frame()
sbert_out_1

In [ ]:
# sbert_text_1 = sbert_out_1['text'].to_string(index=False)
sbert_text_1 = " ".join(sbert_out_1['text'].tolist())
len(sbert_text_1)

In [ ]:
sbert_text_1 = re.sub(r'[\n\\]+', '', sbert_text_1)
sbert_text_1 = re.sub(r'n|n－', '', sbert_text_1)
sbert_text_1

### prompt GPT (part 2)

In [ ]:
summary3 = ''

response = openai.Completion.create(
  model="text-davinci-003",
  prompt= '我剛剛問你：第327次會議中，對於鯨豚保育的重點是什麼？你的答案就是'+ summary1+ '請以之下的資料補充你之前的答案答案'+sbert_text_1,
  temperature=0.9,
  max_tokens=300,
  top_p=1,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=[" Human:", " AI:"]
  )
response
json_object = json.loads(str(response))
text = json_object['choices'][0]['text']
summary3 = summary3 + text

summary3